In [29]:
import numpy as np
import pandas as pd

In [30]:
def load_data():
    raw_df = []
    dict_df = pd.read_excel(r'C:\Users\SR-19\Desktop\Analog_Reports\Solar data\GridTieInverterData.xlsx',skiprows=5,parse_dates={'Time':[2]},index_col='Time',skipfooter=1,sheet_name=None)
    for key,dframe in dict_df.items():
        dframe.drop(dframe.tail(1).index,inplace=True)
        raw_df.append(dframe[['PDC(W)']])
    raw_df=pd.concat(raw_df)
    raw_df.index=pd.to_datetime(raw_df.index,format='%Y-%m-%d %H:%M:%S')
    raw_df.sort_index(inplace=True,ascending=False)
    return raw_df

In [109]:
def load_weather():
    weather = pd.read_csv(r'C:\Users\SR-19\Desktop\Analog_Reports\kanpur_weather_solar.csv',
                             header=26)
    LIST=[]
    for i in range(weather.shape[0]):
        if weather.HR.values[i] < 10:
            string = f"{weather.YEAR.values[i]}-{weather.MO.values[i]}-{weather.DY.values[i]} {0}{weather.HR.values[i]}"
        else:
            string = f"{weather.YEAR.values[i]}-{weather.MO.values[i]}-{weather.DY.values[i]} {weather.HR.values[i]}"

        LIST.append(pd.to_datetime(string,format='%Y-%m-%d %H'))

    weather['time']=LIST
    weather=weather.set_index(weather['time'],drop=False)
    weather.drop(['YEAR','MO','DY','HR','time'],axis=1,inplace=True)
    return weather

In [31]:
raw_df= load_data()

In [32]:
def clean_data(df):
    df['datetime'] = [str(index)[0:13] for index in df.index.values]
    df = df.groupby('datetime').mean()
    df.index=pd.to_datetime(df.index,format='%Y-%m-%d %H')
    return df

In [ ]:
def fill_missing(df):
    start,end=raw_df.index[0],raw_df.index[-1]
    index=pd.date_range(start=start,end=end,freq='1H')
    t_df=pd.DataFrame({'val':np.NaN},index=index)
    result = raw_df.join(t_df, how="outer").drop(['val'],axis=1)
    zero_pow = [19,20,21,22,23,0,1,2,3,4]
    for index in result.index:
        if index.hour in zero_pow:
            result.loc[index]=0
    return result

In [104]:
missing_index=result[result['PDC(W)'].isna()].index.to_list()

In [105]:
dff=result.dropna(axis=0,inplace=False)

In [110]:
weather_df=load_weather()

In [111]:
weather_df

,ALLSKY_SFC_SW_DWN,CLRSKY_SFC_SW_DWN,ALLSKY_SRF_ALB,SZA,ALLSKY_SFC_PAR_TOT,CLRSKY_SFC_PAR_TOT,ALLSKY_SFC_UVA,ALLSKY_SFC_UVB,ALLSKY_SFC_UV_INDEX,T2M,T2MDEW,T2MWET,QV2M,RH2M,PRECTOTCORR,PS,WS10M,WD10M
time,,,,,,,,,,,,,,,,,,
2021-01-01 00:00:00,0.0,0.0,-999.0,-999.0,0.0,0.0,0.0,0.0,0.0,9.80,-2.81,3.50,3.11,40.94,0.0,100.23,2.99,317.86
2021-01-01 01:00:00,0.0,0.0,-999.0,-999.0,0.0,0.0,0.0,0.0,0.0,9.29,-3.38,2.95,2.93,40.50,0.0,100.21,3.01,322.48
2021-01-01 02:00:00,0.0,0.0,-999.0,-999.0,0.0,0.0,0.0,0.0,0.0,8.80,-3.90,2.45,2.81,40.19,0.0,100.19,2.94,325.30
2021-01-01 03:00:00,0.0,0.0,-999.0,-999.0,0.0,0.0,0.0,0.0,0.0,8.38,-4.27,2.06,2.75,40.12,0.0,100.18,2.74,326.99
2021-01-01 04:00:00,0.0,0.0,-999.0,-999.0,0.0,0.0,0.0,0.0,0.0,8.19,-4.61,1.79,2.69,39.62,0.0,100.22,2.45,325.91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-31 19:00:00,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,10.56,9.00,9.78,7.14,90.19,0.0,100.79,2.53,259.70
2021-12-31 20:00:00,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,10.04,8.61,9.32,6.96,90.94,0.0,100.84,2.66,262.58
2021-12-31 21:00:00,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,9.15,7.95,8.55,6.65,92.38,0.0,100.88,2.82,265.71


In [116]:
combine_df = pd.merge(dff, weather_df, left_index=True, right_index=True)

In [118]:
from  sklearn.preprocessing import MinMaxScaler

In [129]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_df = scaler.fit_transform(combine_df.drop(columns=['PDC(W)'],axis=1,inplace=False))

In [130]:
from sklearn.decomposition import PCA
pca = PCA(0.95)
X = scaled_df
X_pca = pca.fit_transform(X)
X_pca.shape

(8416, 5)

In [131]:
pca.explained_variance_ratio_

array([0.48223952, 0.22532658, 0.15862867, 0.05047717, 0.04193721])

In [132]:
input_data = X_pca
target_data = combine_df['PDC(W)'].values
inp_dim = int(0.7*input_data.shape[0])
train_input = input_data[:inp_dim]
train_target = target_data[:inp_dim]
test_input = input_data[inp_dim:]
test_target = target_data[inp_dim:]

In [134]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import explained_variance_score,mean_squared_error,mean_absolute_percentage_error

In [135]:
%%time
base_model = RandomForestRegressor(n_jobs=-1, random_state=42)
base_model.fit(train_input, train_target)

Wall time: 625 ms


RandomForestRegressor(n_jobs=-1, random_state=42)

In [136]:
test_result = base_model.predict(test_input)
train_result = base_model.predict(train_input)
print(f"Training set model score....")
base_model.score(train_input, train_target)

Training set model score....


0.9772773289393092

In [137]:
print(f"Test set model score....")
base_model.score(test_input, test_target)

Test set model score....


0.5815361707548334

In [139]:
def print_summary():
    print(f"test...summary")
    print(f"explained_variance_score ={explained_variance_score(test_result,test_target)}")
    print(f"mean_squared_error ={mean_squared_error(test_result,test_target)}")
    print(f"mean_absolute_percentage_error ={mean_absolute_percentage_error(test_result,test_target)}")
print_summary()

test...summary
explained_variance_score =0.1448330592646021
mean_squared_error =465373.93854636524
mean_absolute_percentage_error =854421817169042.9
